In [4]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
import csv

model=load_model('my_model_4.h5')
tokens=pd.read_csv("embd.csv")
tokens=tokens.iloc[:10001,:]
token={}
for i,j in zip(tokens['keys'],tokens['values']):
    token[i]=j
with open('review-result.csv', 'w', newline = '', encoding = 'utf-8') as f:
        writer = csv.writer(f)
        writer.writerow(['date', 'verified', 'review'])
df = pd.DataFrame()
# selenium functions
def get_url(search_term):
    """Genrate a url from search term"""
    template = "https://www.amazon.com/s?k={}"
    search_term = search_term.replace(" ","+")
    
    #add term query to url
    url = template.format(search_term)
    
    #add page query placeholder
    url += "&page={}"
    
    return url

def extract_record(item):
    """Extract and return data from a single record"""
    
    try:
        #description and url
        atag = item.h2.a
        desc = atag.text.strip()
        url = 'https://www.amazon.com' + atag.get('href')
    except AttributeError:
        return
    
    try:
        #price
        price_parent = item.find('span','a-price')
        price = price_parent.find('span', 'a-offscreen').text
    except AttributeError:
        return
    
    try:
        #rank and rating
        rating = item.i.text
        review_count = item.find('span',{'class': 'a-size-base', 'dir': 'auto'}).text
    except AttributeError:
        rating = ''
        review_count = ''
    
    result = (desc, price, rating, review_count, url)
    return result

def pmain(search_term):
    """Run main program routine"""
    # stratup the webdriver
    options = webdriver.ChromeOptions()
    options.add_argument('headless')
    options.add_argument('window-size=1200x600')
    driver_path = 'chromedriver.exe'
    driver = webdriver.Chrome(driver_path,options=options)
    
    records = []
    url = get_url(search_term)
    
    for page in range(1,2):
        print(page)
        driver.get(url.format(page))
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        results = soup.find_all('div', {'data-component-type': 's-search-result'})
        for itm in results:
            record = extract_record(itm)
            if record:
                records.append(extract_record(itm))
    driver.close()
    #save data to csv file
    with open('result.csv', 'w', newline = '', encoding = 'utf-8') as f:
        writer = csv.writer(f)
        writer.writerow(['desc', 'price', 'rating', 'review_count', 'Url'])
        writer.writerows(records)
# seelenuium function end
def r_get_url(template):
    """Genrate a url from search term"""
    url = template.replace("dp","product-reviews")+'&pageNumber={}'+'&sortBy=recent'
    
    return url

def r_extract_record(item):
    """Extract and return data from a single record"""
    stop = 1
    try:
        #review date
        date = item.find('span',{'data-hook':'review-date'}).text
        if int(date[-4:])<2020:
            stop = 0
        
    except AttributeError:
        return 
    
    try:
        #review
        review = item.find('span',{'data-hook':'review-body'}).text
    except AttributeError:
        return
    
    try:
        #verify purchase
        verified = item.find('span',{'data-hook':'avp-badge'}).text
    except AttributeError:
        return
    
    result = (date, verified, review)
    return result, stop

def r_main(template):
    """Run main program routine"""
    # stratup the webdriver
    options = webdriver.ChromeOptions()
    options.add_argument('headless')
    options.add_argument('window-size=1200x600')
    driver_path = 'chromedriver.exe'
    driver = webdriver.Chrome(driver_path,options=options)
    url = r_get_url(template)
    records = []
    stop = 1
    page=0
    for i in range(2):
        print(i)
        page+=1
        driver.get(url.format(page))
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        results = soup.find_all('div', {'data-hook': 'review'})
        for itm in results:
            r = r_extract_record(itm)
            
            if r:
                stop = r[-1]
                records.append(r[0])
    driver.close()
    
    #save data to csv file
    with open('review-result.csv', 'a', newline = '', encoding = 'utf-8') as f:
        writer = csv.writer(f)
        writer.writerows(records)
def predict_output(X):
    X=X.lower()
    max_length=120
    trunc_type='post'
    x_predict=[[token[i] if i in token else 1 for i in X.split()]]
    padded=pad_sequences(x_predict,maxlen=max_length,truncating=trunc_type)
    return "Positive" if model.predict_classes(padded)[0] == 1 else "Negative"
pmain("smart phone")
df=pd.read_csv("result.csv")
for i in range(min(10,df.shape[0])):
    r_main(df["Url"][i])
        
    
    #validate input hours
dfreview=pd.read_csv('review-result.csv')  
countp=0

for i in range(dfreview.shape[0]):
    output = predict_output(dfreview.review[i])
    if(output=="Positive"):
        countp=countp+1
countn=dfreview.shape[0]-countp

1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1


In [4]:
dfreview=pd.read_csv('review-result.csv')  
countp=0
for i in range(dfreview.shape[0]):
    output = predict_output(dfreview.review[i])
    if(output=="Positive"):
        countp=countp+1
countn=dfreview.shape[0]-countp


AttributeError: 'str' object has no attribute 'shape'

In [5]:
dfreview

,date,verified,review
0,"Reviewed in the United States on November 2, 2020",Verified Purchase,\n\n I was really happy with the phone for a ...
1,"Reviewed in the United States on November 3, 2020",Verified Purchase,"\n\n I loved the phone, and it worked really ..."
2,"Reviewed in the United States on November 17, ...",Verified Purchase,\n\n This phone does have a 4500 mAh battery....
3,"Reviewed in the United States on November 30, ...",Verified Purchase,"\n\n After several weeks of use, I can confid..."
4,"Reviewed in the United States on November 2, 2020",Verified Purchase,\n\n I was really happy with the phone for a ...
...,...,...,...
253,"Reviewed in the United States on October 25, 2020",Verified Purchase,\n\n I’ve bought this for testing apps on And...
254,"Reviewed in the United States on September 8, ...",Verified Purchase,\n\n I received my order today\n\n
255,"Reviewed in the United States on September 27,...",Verified Purchase,"\n\n Decent phone, still can't find a back bu..."
256,"Reviewed in the United States on September 23,...",Verified Purchase,\n\n It's okay the only main concern is somet...


In [6]:
dfreview.shape[0]

258

In [8]:
dfreview['review'][0]

'\n\n  I was really happy with the phone for a day until I checked the battery capacity. The battery was supposed to be 4500 mAh. But instead I got a 3300 mAh battery! Am I scammed!? I want a refund! Please reply.\n\n'

In [12]:
model.predict(dfreview['review'][0])

AttributeError: 'str' object has no attribute 'shape'

In [4]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
import csv

In [5]:
model=load_model('my_model_4.h5')

In [6]:
dfreview=pd.read_csv('review-result.csv')  
countp=0
for i in range(dfreview.shape[0]):
    output = model.predict(dfreview['review'][i])
    if(output=="Positive"):
        countp=countp+1
countn=dfreview.shape[0]-countp


AttributeError: 'str' object has no attribute 'shape'

In [14]:
def predict_output(X):
    X=X.lower()
    x_predict=[[token[i] if i in token else 1 for i in X.split()]]
    padded=pad_sequences(x_predict,maxlen=max_length,truncating=trunc_type)
    return "Positive" if model.predict_classes(padded)[0] == 1 else "Negative"
for i in range(dfreview.shape[0]):
    output = predict_output(dfreview.review[i])
    if(output=="Positive"):
        countp=countp+1
countn=dfreview.shape[0]-countp

In [13]:
max_length=120
trunc_type='post'
model=load_model('my_model_4.h5')
tokens=pd.read_csv("embd.csv")
tokens=tokens.iloc[:10001,:]
token={}
for i,j in zip(tokens['keys'],tokens['values']):
    token[i]=j

In [5]:
print(countp,countn)

106 52
